In [ ]:
from pprint import pprint

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 500

%matplotlib inline
df_clients = pd.read_csv('clients.csv', index_col='client_id')
df_train = pd.read_csv('uplift_train.csv', index_col='client_id')
df_test = pd.read_csv('uplift_test.csv', index_col='client_id')


df_features = df_clients.copy()
df_features['first_issue_time'] = \
    (pd.to_datetime(df_features['first_issue_date'])
     - pd.to_datetime(df_features['first_issue_date']).min()) / pd.Timedelta('365d')

df_features['first_redeem_time'] = \
    (pd.to_datetime(df_features['first_redeem_date'])
     - pd.to_datetime(df_features['first_redeem_date']).min()) / pd.Timedelta('365d')

df_features['issue_redeem_delay'] = df_features['first_redeem_time'] \
    - df_features['first_issue_time']

df_features = df_features.join(pd.get_dummies(df_features['gender']))
df_features['first_redeem_time'] = df_features['first_redeem_time'].fillna(df_features['first_redeem_time'].mean())
df_features['issue_redeem_delay'] = df_features['issue_redeem_delay'].fillna(df_features['issue_redeem_delay'].mean())

df_features = df_features.drop(['first_issue_date', 'first_redeem_date', 'gender'], axis=1)

indices_train = df_train.index
indices_test = df_test.index
indices_learn, indices_valid = train_test_split(df_train.index, test_size=0.3, random_state=123)


X_train = df_features.loc[indices_learn, :]
y_train = df_train.loc[indices_learn, 'target']
treat_train = df_train.loc[indices_learn, 'treatment_flg']

X_val = df_features.loc[indices_valid, :]
y_val = df_train.loc[indices_valid, 'target']
treat_val =  df_train.loc[indices_valid, 'treatment_flg']

X_train_full = df_features.loc[indices_train, :]
y_train_full = df_train.loc[:, 'target']
treat_train_full = df_train.loc[:, 'treatment_flg']

X_test = df_features.loc[indices_test, :]

cat_features = ['gender']

In [ ]:
from torch.utils.data import Dataset, DataLoader

class X5Dataset(Dataset):

    def __init__(self, data, target=None, treatment=None):
        super(X5Dataset, self).__init__()
        self.data = torch.from_numpy(data.values).type(torch.FloatTensor)
        if target is not None:
            self.target = torch.from_numpy(target.values).type(torch.FloatTensor)
        if treatment is not None:
            self.treatment = torch.from_numpy(treatment.values).type(torch.FloatTensor)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        m0 = torch.zeros((1, self.data.shape[1] + 1))
        m0[:, :-1] = self.data[idx, :]
        m0[:, -1] = 0
        m1 = torch.zeros((1, self.data.shape[1] + 1))
        m1[:, :-1] = self.data[idx, :]
        m1[:, -1] = 1

        if self.target is None:
            return m0.squeeze().to(device), m1.squeeze().to(device)
        else:
            return (m0.squeeze().to(device), m1.squeeze().to(device), 
                    self.target[idx].to(device), self.treatment[idx].to(device))

In [ ]:
train_dataset = X5Dataset(X_train, y_train, treat_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = X5Dataset(X_val, y_val, treat_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch
import torch.nn as nn

#class SMITEEncoder(nn.Model):

#    def __init__(self, input_dim, hidden_dims):
#        super(SMITEEncoder, self).__init__()
#        self.net = 

#    def init_base_network(self, dims):
#        layers = []
#        for i in range(len(dims) - 1):
#            layers.append(nn.Linear(dims[i], dims[i+1]))
#            layers.append(nn.ReLU())
#        return nn.Sequential(*layers[:-1])

p = 0.5


def loss_function(y_true, preds0, preds1, alpha, T, p):
    z = y_true * (T - p) / (p * (1 - p))
    vec = (z - preds1 + preds0)
    J = vec.matmul(vec) / len(vec)
    preds = T * preds1 + (1 - T) * preds0
    L = nn.functional.binary_cross_entropy(preds, y_true)
    return alpha * L + (1 - alpha) * J

input_dim = 8
hidden_dims = [256, 256, 256, 256, 256, 256, 256]
dims = [input_dim] + hidden_dims + [1]

layers = []
for i in range(len(dims) - 1):
    layers.append(nn.Linear(dims[i], dims[i+1]))
    layers.append(nn.ReLU())
layers[-1] = nn.Sigmoid()
model = nn.Sequential(*layers).to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), 0.001)

In [ ]:
N_EPOCHS = 100
alpha = 0.1
p = 0.5

for i in range(N_EPOCHS):

    loss_train = []

    for data0, data1, target, treat in train_loader:
        
        preds0 = model(data0).squeeze()
        preds1 = model(data1).squeeze()
        loss = loss_function(target, preds0, preds1, alpha, treat, p)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        loss_train.append(loss.item())
    loss_train = np.mean(loss_train)

    labels_val = []
    preds_val = []
    loss_val = []
    for data0, data1, target, treat in val_loader:
        labels_val += list(target)
        preds0 = model(data0).squeeze()
        preds1 = model(data1).squeeze()
        loss = loss_function(target, preds0, preds1, alpha, treat, p)
        preds_val += list(preds1 - preds0)
        loss_val.append(loss.item())

    accuracy = (np.array(preds_val).astype(np.int16) == np.array(labels_val)).sum() / len(preds_val)
    print(f'Epoch {i} \t Loss train {loss_train:.4f} \t Accuracy test {accuracy:4f} \t Loss test {np.mean(loss_val):.4f}')

Epoch 0 	 Loss train 2.2962 	 Accuracy test 0.379507 	 Loss test 2.2968
Epoch 1 	 Loss train 2.2950 	 Accuracy test 0.379507 	 Loss test 2.2967
Epoch 2 	 Loss train 2.2965 	 Accuracy test 0.379507 	 Loss test 2.2968
Epoch 3 	 Loss train 2.2961 	 Accuracy test 0.379507 	 Loss test 2.2968
Epoch 4 	 Loss train 2.2949 	 Accuracy test 0.379507 	 Loss test 2.2955
Epoch 5 	 Loss train 2.2980 	 Accuracy test 0.379507 	 Loss test 2.2970
Epoch 6 	 Loss train 2.2977 	 Accuracy test 0.379507 	 Loss test 2.2967
Epoch 7 	 Loss train 2.2963 	 Accuracy test 0.379507 	 Loss test 2.2967
Epoch 8 	 Loss train 2.2981 	 Accuracy test 0.379507 	 Loss test 2.2967
Epoch 9 	 Loss train 2.2981 	 Accuracy test 0.379507 	 Loss test 2.2956
Epoch 10 	 Loss train 2.2968 	 Accuracy test 0.379507 	 Loss test 2.2964
Epoch 11 	 Loss train 2.2954 	 Accuracy test 0.379507 	 Loss test 2.2956
Epoch 12 	 Loss train 2.2973 	 Accuracy test 0.379507 	 Loss test 2.2964
Epoch 13 	 Loss train 2.2947 	 Accuracy test 0.379507 	 Loss 

In [ ]:
import re
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras.utils import to_categorical

Using TensorFlow backend.


In [ ]:
def build_base_network(input_shape):
    
    seq = Sequential()

    #flatten 
    seq.add(Dense(128, activation='relu'))
    seq.add(Dense(128, activation='relu'))
    seq.add(Dense(50, activation='relu'))
    return seq

In [ ]:
input_dim = (8, )
input_a = Input(shape=input_dim)
input_b = Input(shape=input_dim)

base_network = build_base_network(input_dim)
feat_vecs_a = base_network(input_a)
feat_vecs_b = base_network(input_b)

In [ ]:
# def output_uplift(vects):
#     m0, m1 = vects
    
#     return m0, m1


# def uplift_output_shape(shapes):
#     shape1, shape2 = shapes
#     return (shape1[0], 2)

# uplift = Lambda(output_uplift, output_shape=uplift_output_shape)([feat_vecs_a, feat_vecs_b])

In [ ]:
epochs = 2
rms = RMSprop()
output = Dense(1, activation='sigmoid')(feat_vecs_a)

model = Model(input=[input_a, input_b], output=[output, output])

In [ ]:
def smite_loss(y_true, y_pred):
    m0, m1 = y_pred[0], y_pred[1]
    t = treat_train
    mt = K.dot(t, m1) + K.dot((1 - t), m0)
    z = [FlaggedFloat(t * o / 0.5 - (1 - t) * o / 0.5) for o, t in zip(y_true, t)]
    J = K.mean(K.square(z - (m1 - m0)))
    L = K.sum(y_true * K.log(mt) + (1 - y_true) * K.log(1 - mt))
    return (1 - 0.2) * J + 0.2 * L 

model.compile(loss=smite_loss, optimizer=rms)

Tensor("loss_3/dense_4_loss/smite_loss/strided_slice_1:0", shape=(), dtype=float32)


AttributeError: 'tuple' object has no attribute 'rank'

In [ ]:
sample_1 = X_train.assign(treatment = 1)
sample_2 = X_train.assign(treatment = 0) 

model.fit([sample_1, sample_2], y_train, validation_split=.25, batch_size=1, verbose=2, nb_epoch=epochs)

In [ ]:
model.predict([X_test.assign(treatment = 1), X_test.assign(treatment = 0)])